In [228]:
# !pip install prefigure

In [227]:
import torch
from torch import nn
import torchaudio
from torchaudio import transforms as T
from glob import glob
import random
import os
from IPython.display import Audio
# from prefigure.prefigure import get_all_args, push_wandb_config

In [82]:
class PadCrop(nn.Module):
    def __init__(self, n_samples):
        super().__init__()
        self.n_samples = n_samples
#         self.randomize = randomize

    def __call__(self, signal):
        n, s = signal.shape
        start = torch.randint(0, max(0, s - self.n_samples) + 1, []).item()
        end = start + self.n_samples
        output = signal.new_zeros([n, self.n_samples])
        output[:, :min(s, self.n_samples)] = signal[:, start:end]
        return output


class RandomPhaseInvert(nn.Module):
    def __init__(self, p=0.5):
        super().__init__()
        self.p = p

    def __call__(self, signal):
        return -signal if (random.random() < self.p) else signal


class Stereo(nn.Module):
    def __call__(self, signal):
        signal_shape = signal.shape
        # Check if it's mono
        if len(signal_shape) == 1:  # s -> 2, s
            signal = signal.unsqueeze(0).repeat(2, 1)
        elif len(signal_shape) == 2:
            if signal_shape[0] == 1:  # 1, s -> 2, s
                signal = signal.repeat(2, 1)
            elif signal_shape[0] > 2:  # ?, s -> 2,s
                signal = signal[:2, :]

        return signal


In [294]:
# Changing global_args.samplesize
# Changing global_args ---- will be something else
training_dir = '/Users/cameronolson/ML-DataSets/Sample Dataset'
sample_size = 44100
sample_rate = 44100




class DrumDataset(torch.utils.data.Dataset):
    def __init__(self, paths, sample_size, sample_rate):
        super().__init__()
        # Create the bucket for files
        self.filenames = []

        # augment data: PadCrop, RandomPhaseInvert
        # Look at PadCrop --- is this shortening or just for random
        self.augs = torch.nn.Sequential(
            PadCrop(sample_size),  # Probobly get rid of Pad crop!!!!!!
            RandomPhaseInvert(),
        )
        # Re Channel data into stereo()
        self.encoding = torch.nn.Sequential(
                                            Stereo())
        
        file_type = '.wav'
        # Put file names in the bucket
        for path in paths:
            self.filenames += glob(f'{path}/**/*{file_type}', recursive=True)  # recursive?

        self.sr = sample_rate

    # Load audio file --- torchaudio
    def load_file(self, filename):

        audio, sr = torchaudio.load(filename)
        # Resample audio files
        if sr != self.sr:
            resample_tf = T.Resample(sr, self.sr)
            resample_tf(audio)
        return audio

    # Return length of dataset
    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        # Audio filename
        audio_filename = self.filenames[idx]

        # Load the audio file
        audio = self.load_file(audio_filename)

        # augmentation
        audio = self.augs(audio)
        
        audio = self.encoding(audio)

        # Normalize
        audio = audio.clamp(-1, 1)

        return audio, audio_filename

## What am I confused on?

This is important
'https://pypi.org/project/prefigure/'

I am confused on 
- How to get data in 
- How to pass args in 
- What is __call__

In [295]:
len(train_set)

17314

In [296]:
# train_set = DrumDataset([args.training_dir], args)
train_set = DrumDataset([training_dir], sample_size, sample_rate)




In [306]:
waveform, file = train_set[990]
waveform
file

'/Users/cameronolson/ML-DataSets/Sample Dataset/hats/ClosedHH Eliot 1 V121.wav'

In [307]:
import tempfile
def inspect_file(path):
    print("-" * 10)
    print("Source:", path)
    print("-" * 10)
    print(f" - File size: {os.path.getsize(path)} bytes")
    print(f" - {torchaudio.info(path)}")
    print()
    

with tempfile.TemporaryDirectory() as tempdir:
    path = f"{tempdir}/save_example.{'wav'}"
    torchaudio.save(path, waveform, sample_rate, format='wav')
    inspect_file(path)
    waveform, sample_rate = torchaudio.load(path)
Audio(waveform.numpy()[0], rate=sample_rate)

----------
Source: /var/folders/s1/bp0h8p1s16j30697srtjp8_w0000gn/T/tmpmaiwccnb/save_example.wav
----------
 - File size: 352858 bytes
 - AudioMetaData(sample_rate=44100, num_frames=44100, num_channels=2, bits_per_sample=32, encoding=PCM_F)



In [301]:


# waveform, sample_rate = torchaudio.load(path)
# Audio(waveform.numpy()[0], rate=sample_rate)

# TODO ____________________

Essential to understand
- How to pass in the global args into the model from the command line

TODO
- Build notebook back into script for training
- Study Dance Diffusion: how they enitialize it



In [1]:
44100//2

22050